In [60]:
# Created 2017, Zack Gainsforth
%pylab osx
import sys, os
import QuickPlot
import matplotlib.pylab as pylab
# pylab.rcParams['figure.figsize'] = 8, 6  # that's default image size for this interactive session
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# from ipywidgets.widgets import interactive, fixed, interact
%config InlineBackend.figure_format = 'retina'

from scipy.optimize import nnls

from tabulate import tabulate

Populating the interactive namespace from numpy and matplotlib


/Users/Zack/anaconda/envs/conda36/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['table']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [63]:
ICPNames = ['CrKa', 'CrKb', 'FeKa', 'FeKb', 'MgK', 'TiKa', 'VKa', 'OK']
#ICPNames = ['CrKa', 'CrKb', 'FeKa', 'FeKb', 'OK']

# Now read all the ICPs in and print them into a figure.
ICPs = []
plt.figure('ICPs')
plt.subplot(3, len(ICPNames), 1)
for i, n in enumerate(ICPNames):
    ICPs.append(genfromtxt(n+'.txt'))
    plt.subplot(3, len(ICPNames), i+1)
    plt.imshow(ICPs[i])
    plt.title(ICPNames[i])
      
# Define basis vectors and put them into the solution vector.
Oct = ICPs[0]
#Tet = ICPs[2]
Tet = ICPs[4]
Ox = ICPs[-1]

# Make a column matrix of the basis vectors.
X = np.zeros((len(ravel(Oct)), 3))
X[:,0] = ravel(Oct)
X[:,1] = ravel(Tet)
X[:,2] = ravel(Ox)

def DoTheFit(ICPNames, ICPs, X, row):
    table = []
    for i, n in enumerate(ICPNames):
        Answer = nnls(X,ravel(ICPs[i]))
        Reconstruction = dot(X, Answer[0])
        deco = Answer[0]/norm(Answer[0])*100
        res = Answer[1]
        counts = sum(sum(ICPs[i]))
        table.append([n, deco[0], deco[1], deco[2], res, counts, sqrt(res)/sqrt(counts)])
        plt.subplot(3, len(ICPNames), i+1+((row-1)*len(ICPNames)))
        #plt.imshow(Reconstruction.reshape((31,31)))
        plt.imshow(ICPs[i])

        

                


        
    print(tabulate(table, headers=['ICP', 'Oct', 'Tet', 'Ox', 'Residual', 'Total counts', 'sigma'], floatfmt='0.02f', tablefmt='grid'))
    
print('Direct data')
DoTheFit(ICPNames, ICPs, X, 2)

# Try subtracting off beta intensity from overlap peaks.
ICPs[1] -= ICPs[0]*0.123 # Cr on Mn
ICPs[3] -= ICPs[2]*0.125 # Fe on Co
ICPs[6] -= ICPs[5]*0.125 # Ti overlap on V

print()
print('After subtracting off overlaps')
DoTheFit(ICPNames, ICPs, X, 3)



Direct data
+-------+--------+--------+--------+------------+----------------+---------+
| ICP   |    Oct |    Tet |     Ox |   Residual |   Total counts |   sigma |
+=======+========+========+========+============+================+=========+
| CrKa  | 100.00 |   0.00 |   0.00 |       0.00 |    14556985.31 |    0.00 |
+-------+--------+--------+--------+------------+----------------+---------+
| CrKb  |  50.48 |  86.32 |   0.00 |    1804.77 |     1855787.30 |    0.03 |
+-------+--------+--------+--------+------------+----------------+---------+
| FeKa  |   1.29 |  99.93 |   3.45 |   12001.36 |     6553299.86 |    0.04 |
+-------+--------+--------+--------+------------+----------------+---------+
| FeKb  |   2.18 |  99.94 |   2.51 |    2004.92 |      877545.53 |    0.05 |
+-------+--------+--------+--------+------------+----------------+---------+
| MgK   |   0.00 | 100.00 |   0.00 |       0.00 |      352591.16 |    0.00 |
+-------+--------+--------+--------+------------+---------------

## 